# Classificador de Risco com TF-IDF
Este notebook treina um classificador simples para identificar frases de alto ou baixo risco com base em descrições de sintomas.

### Objetivos
- Carregar um conjunto de frases rotuladas com níveis de risco.
- Transformar o texto em vetores TF-IDF.
- Treinar um modelo de regressão logística e avaliar o desempenho.
- Testar o modelo com novas frases para validar o comportamento.

In [5]:
from pathlib import Path

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [6]:
BASE_DIR = Path.cwd().parents[0]
data_path = BASE_DIR / "data" / "risk_classification.csv"

if not data_path.exists():
    raise FileNotFoundError(f"Arquivo não encontrado: {data_path}")

raw_df = pd.read_csv(data_path)
raw_df.head()

frase     situacao
0  sinto dor intensa no peito acompanhada de suor...   alto risco
1  tive um leve incômodo nas costas após trabalha...  baixo risco
2  falta de ar súbita e dor que irradia para o br...   alto risco
3        leve dor de cabeça que melhora com descanso  baixo risco
4  palpitações fortes e tontura quando tento subi...   alto risco

### Exploração Rápida
Verificamos o balanceamento da base simulada e, em seguida, identificamos as principais palavras para cada classe a partir dos coeficientes da regressão logística.

In [7]:
raw_df['situacao'].value_counts()


situacao
alto risco     10
baixo risco    10
Name: count, dtype: int64

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    raw_df["frase"], raw_df["situacao"], test_size=0.3, random_state=42, stratify=raw_df["situacao"]
)

pipeline = Pipeline(
    steps=[
        ("tfidf", TfidfVectorizer(ngram_range=(1, 2), min_df=1)),
        ("clf", LogisticRegression(max_iter=1000, class_weight="balanced")),
    ]
)

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
                ('clf',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

In [9]:
y_pred = pipeline.predict(X_test)
print(f"Acurácia: {pipeline.score(X_test, y_test):.2f}")
print("\nRelatório de classificação:\n")
print(classification_report(y_test, y_pred))

print("Matriz de confusão:")
print(confusion_matrix(y_test, y_pred))

Acurácia: 1.00

Relatório de classificação:

              precision    recall  f1-score   support

  alto risco       1.00      1.00      1.00         3
 baixo risco       1.00      1.00      1.00         3

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

Matriz de confusão:
[[3 0]
 [0 3]]


### Principais Termos por Classe
A regressão logística permite interpretar pesos dos termos. Valores positivos indicam associação com `alto risco`, enquanto negativos sugerem `baixo risco`.

In [10]:
vectorizer = pipeline.named_steps['tfidf']
clf = pipeline.named_steps['clf']
feature_names = vectorizer.get_feature_names_out()
coefs = clf.coef_[0]
top_alto = coefs.argsort()[-10:][::-1]
top_baixo = coefs.argsort()[:10]
print('Termos mais associados a ALTO risco:')
for idx in top_alto:
    print(f'  {feature_names[idx]} - peso {coefs[idx]:.3f}')

print(
    'Termos mais associados a BAIXO risco:'
)
for idx in top_baixo:
    print(f'  {feature_names[idx]} - peso {coefs[idx]:.3f}')


Termos mais associados a ALTO risco:
  leve - peso 0.312
  tosse - peso 0.232
  do - peso 0.193
  após - peso 0.172
  irritando - peso 0.149
  garganta - peso 0.149
  tosse passageira - peso 0.149
  irritando garganta - peso 0.149
  passageira irritando - peso 0.149
  passageira - peso 0.149
Termos mais associados a BAIXO risco:
  dor - peso -0.242
  de - peso -0.196
  suor - peso -0.184
  suor frio - peso -0.184
  frio - peso -0.184
  falta - peso -0.183
  de ar - peso -0.183
  ar - peso -0.183
  falta de - peso -0.183
  no - peso -0.142


### Observações sobre Viés
Por se tratar de uma base sintética e pequena, a cobertura de vocabulário é limitada. Termos muito específicos foram rotulados manualmente; portanto, recomenda-se expandir a base antes de utilizar o modelo em produção.

In [11]:
novas_frases = [
    "sinto dor no peito e falta de ar depois de subir escadas",
    "leve desconforto no ombro após alongamento",
    "visão turva, pressão alta e suor frio na madrugada",
]

predicoes = pipeline.predict(novas_frases)

for frase, risco in zip(novas_frases, predicoes):
    print(f"Frase: {frase}\n → Risco previsto: {risco}\n")

Frase: sinto dor no peito e falta de ar depois de subir escadas
 → Risco previsto: alto risco

Frase: leve desconforto no ombro após alongamento
 → Risco previsto: baixo risco

Frase: visão turva, pressão alta e suor frio na madrugada
 → Risco previsto: alto risco

